In [4]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, PossessionCondition, ConditionalExplanation, CompositeExplanation

In [5]:
# Example usage:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2], score_child=child1)

In [6]:
knowledgebase = ArgumentationFramework()

knowledgebase.add_adjective( 
    BooleanAdjective("leaf")
)

knowledgebase.add_adjective( 
    PointerAdjective("score",

    explanation = ConditionalExplanation(
        condition = PossessionCondition("leaf"),
        true_explanation = Assumption("Leaf nodes have scores from the evaluation function"),
        false_explanation = CompositeExplanation(
            Assumption("Internal nodes have scores from children"),
            Possession("backtracing child"))
    ))
)

knowledgebase.add_adjective(
    BooleanAdjective("opponent player turn")
)

knowledgebase.add_adjective( 
    PointerAdjective("backtracing child",

    explanation = ConditionalExplanation(
        condition = PossessionCondition("opponent player turn"),
        true_explanation = CompositeExplanation(
            Assumption("We assume the opponent will do their best move."),
            Possession("backtracing child", "minoptimal")),
        false_explanation = CompositeExplanation(
            Assumption("On our turn we take the maximum rated move."),
            Possession("backtracing child", "maxoptimal"))
    ))
)

knowledgebase.add_adjective(
    ComparisonAdjective("better", "score", ">")
)

knowledgebase.add_adjective( 
    PointerAdjective("siblings")
)

knowledgebase.add_adjective(
    MaxRankAdjective("maxoptimal", "better", "siblings")
)
knowledgebase.add_adjective(
    MinRankAdjective("minoptimal", "better", "siblings")
)

In [7]:
#print(knowledgebase)

In [8]:
explainer = ArgumentativeExplainer(knowledgebase)
explainer.set_getter("siblings", lambda node: [sibling for sibling in node.parent.children if sibling is not node])
explainer.set_getter("leaf", lambda node: node.is_leaf)
explainer.set_getter("backtracing child", lambda node: node.score_child)
explainer.set_getter("score", lambda node: node.score)
explainer.set_getter("opponent player turn", lambda node: not node.maximizing_player_turn)
# alternative getter for backtracing child:
# lambda node: min(node.children, key=lambda child: child.score),

explainer.set_tree_search_motivation(lambda root: root.children, "maxoptimal")

Try

In [9]:
# Generate explanations
print(explainer.explain_adjective(child1, "maxoptimal"))


A node is max-ranked in a RankingAdjective if the Ranking Condition is TRUE when compared to all its siblings
 ∧ By definition of leaf → ¬(child1 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player turn → child1 is opponent player turn
 ∧ We assume the opponent will do their best move.
 ∧ A node is min-ranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings
 ∧ By definition of leaf → leaf11 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf11 has score = 3
 ∧ By definition of leaf → leaf12 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf12 has score = 4
 ∧ By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12) → leaf11 is minoptimal → child1 has backtracing child = leaf11 → child1 has score = 3
 ∧ By definition of leaf → ¬(child2 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player t

In [10]:
print(explainer.explain_adjective(child1, "minoptimal"))

A node is min-ranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings
 ∧ By definition of leaf → ¬(child1 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player turn → child1 is opponent player turn
 ∧ We assume the opponent will do their best move.
 ∧ A node is min-ranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings
 ∧ By definition of leaf → leaf11 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf11 has score = 3
 ∧ By definition of leaf → leaf12 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf12 has score = 4
 ∧ By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12) → leaf11 is minoptimal → child1 has backtracing child = leaf11 → child1 has score = 3
 ∧ By definition of leaf → ¬(child2 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player 

In [11]:
print(explainer.explain_adjective(child2, "maxoptimal"))

A node is max-ranked in a RankingAdjective if the Ranking Condition is TRUE when compared to all its siblings
 ∧ By definition of leaf → ¬(child2 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player turn → child2 is opponent player turn
 ∧ We assume the opponent will do their best move.
 ∧ A node is min-ranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings
 ∧ By definition of leaf → leaf22 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf22 has score = 2
 ∧ By definition of leaf → leaf21 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf21 has score = 8
 ∧ By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf22 is better than leaf21) → leaf22 is minoptimal → child2 has backtracing child = leaf22 → child2 has score = 2
 ∧ By definition of leaf → ¬(child1 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player t

In [12]:
print(explainer.explain_adjective(root, "backtracing child"))

By definition of opponent player turn → ¬(root is opponent player turn)
 ∧ On our turn we take the maximum rated move.
 ∧ A node is max-ranked in a RankingAdjective if the Ranking Condition is TRUE when compared to all its siblings
 ∧ By definition of leaf → ¬(child1 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player turn → child1 is opponent player turn
 ∧ We assume the opponent will do their best move.
 ∧ A node is min-ranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings
 ∧ By definition of leaf → leaf11 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf11 has score = 3
 ∧ By definition of leaf → leaf12 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf12 has score = 4
 ∧ By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12) → leaf11 is minoptimal → child1 has backtracing child = leaf11 → child1 has score = 3
 ∧ B

In [13]:
print(explainer.explain_adjective(root, "score"))

By definition of leaf → ¬(root is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player turn → ¬(root is opponent player turn)
 ∧ On our turn we take the maximum rated move.
 ∧ A node is max-ranked in a RankingAdjective if the Ranking Condition is TRUE when compared to all its siblings
 ∧ By definition of leaf → ¬(child1 is leaf)
 ∧ Internal nodes have scores from children
 ∧ By definition of opponent player turn → child1 is opponent player turn
 ∧ We assume the opponent will do their best move.
 ∧ A node is min-ranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings
 ∧ By definition of leaf → leaf11 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf11 has score = 3
 ∧ By definition of leaf → leaf12 is leaf
 ∧ Leaf nodes have scores from the evaluation function → leaf12 has score = 4
 ∧ By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12) → l

Track down that Not before the "Considering your definition of leaf"

In [14]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

None
